<a href="https://colab.research.google.com/github/JakeOh/202205_itw_bd34/blob/main/ml03_train_test_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   훈련 셋과 테스트 셋을 나누는 방법
    *   순차적 추출(sequential sampling)
    *   임의 추출(random sampling)
    *   층화 추출(stratified sampling)
*   분류 모델 평가 지표(metrics)
    *   정확도(accuracy)
    *   정밀도(precision)
    *   재현율(recall)
    *   F1-score
*   특성 스케일링(feature scaling)
    *   표준화(standardization)
    *   정규화(normalization)

# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier  # KNN 분류기(모델)
from sklearn.model_selection import train_test_split  # 훈련/테스트 셋 분리 함수
from sklearn.metrics import confusion_matrix, classification_report  # 모델 평가 지표
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # 변환기(특성 스케일링)

# 데이터 준비

In [2]:
# fish 데이터 셋
fish_csv = 'https://github.com/rickiepark/hg-mldl/raw/master/fish.csv'

In [3]:
fish = pd.read_csv(fish_csv)

In [4]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


*   이진 분류: Bream(도미) vs Smelt(빙어)
*   독립변수: Weight(무게), Length(길이)

In [8]:
# 데이터 - 물고기의 무게와 길이를 저장한 2차원 배열
X = fish.loc[fish.Species.isin(['Bream', 'Smelt']), ['Weight', 'Length']].values

In [9]:
X.shape

(49, 2)

In [10]:
X[:5]

array([[242. ,  25.4],
       [290. ,  26.3],
       [340. ,  26.5],
       [363. ,  29. ],
       [430. ,  29. ]])

In [23]:
# 타겟(레이블, 종속변수, 관심변수) - 물고기 종류를 저장한 1차원 배열
y = fish.loc[fish.Species.isin(['Bream', 'Smelt']), 'Species'].values

In [24]:
y.shape

(49,)

In [25]:
y

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt'],
      dtype=object)

# 훈련 셋/테스트 셋 분리

## 순차 추출(sequential sampling)

In [27]:
# 훈련 셋과 테스트 셋의 비율을 7:3 비율로 나눔.
num_trains = 35  # 훈련 셋의 샘플 개수

In [28]:
X_train = X[:num_trains]  # 훈련 셋
X_test = X[num_trains:]   # 테스트 셋
y_train = y[:num_trains]  # 훈련 레이블
y_test = y[num_trains:]   # 테스트 레이블

In [29]:
X_train.shape, X_test.shape

((35, 2), (14, 2))

In [30]:
X_train[:5]

array([[242. ,  25.4],
       [290. ,  26.3],
       [340. ,  26.5],
       [363. ,  29. ],
       [430. ,  29. ]])

In [31]:
X_test[:5]

array([[ 6.7,  9.8],
       [ 7.5, 10.5],
       [ 7. , 10.6],
       [ 9.7, 11. ],
       [ 9.8, 11.2]])

In [32]:
y_train.shape, y_test.shape

((35,), (14,))

In [33]:
y_train

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream'],
      dtype=object)

In [34]:
y_test

array(['Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt'],
      dtype=object)

In [35]:
np.unique(y_train, return_counts=True)
#> return: y_train 배열에서 unique한 값들로 이루어진 배열, unique한 값들의 개수들의 배열.

(array(['Bream'], dtype=object), array([35]))

In [36]:
np.unique(y_test, return_counts=True)

(array(['Smelt'], dtype=object), array([14]))

특성 행렬(데이터)이 무작위로 섞여있지 않고 물고기 종류에 대해서 정렬된 상태였기 때문에, 훈련 셋과 훈련 레이블에는 도미(Bream)만 선택됐고, 테스트 셋과 테스트 레이블에서는 빙어(Smelt)만 선택됨.  --> __샘플링 편향(sampling bias)__

### KNN 모델을 훈련, 평가

In [37]:
knn = KNeighborsClassifier()  # 모델 생성

In [38]:
knn.fit(X_train, y_train)  # 모델 훈련

KNeighborsClassifier()

In [39]:
train_acc = knn.score(X_train, y_train)  # 훈련 셋에서의 정확도
train_acc

1.0

In [40]:
test_acc = knn.score(X_test, y_test)  # 테스트 셋에서의 정확도
test_acc

0.0

In [41]:
knn.predict(X_test)  # 테스트 셋의 예측값

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream'],
      dtype=object)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                 train_size=35,  # 훈련 셋의 샘플 개수
                 shuffle=False)  # 데이터를 무작위로 섞을 것인 지.
# shuffle=False: 순차 추출(sequential sampling)

In [44]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((35, 2), (14, 2), (35,), (14,))

In [45]:
np.unique(y_train, return_counts=True)

(array(['Bream'], dtype=object), array([35]))

In [46]:
np.unique(y_test, return_counts=True)

(array(['Smelt'], dtype=object), array([14]))

## 임의 추출(random sampling)

In [50]:
idx = np.arange(49)
print(idx)

np.random.seed(1)
np.random.shuffle(idx)  # 배열 idx를 무작위로 섞음.
print(idx)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48]
[27 34 39 48  2  3 42 29 45 30 31 38 21 35 19 41 36 26 22 13 40 17 44 24
 23  4 32 14 10 28 47 33 18 20 25  6  7 46  1 16  0 15  5 11  9  8 12 43
 37]


In [51]:
train_idx = idx[:35]
print(train_idx)
test_idx = idx[35:]
print(test_idx)

[27 34 39 48  2  3 42 29 45 30 31 38 21 35 19 41 36 26 22 13 40 17 44 24
 23  4 32 14 10 28 47 33 18 20 25]
[ 6  7 46  1 16  0 15  5 11  9  8 12 43 37]


In [52]:
X_train = X[train_idx]
y_train = y[train_idx]

In [53]:
np.unique(y_train, return_counts=True)

(array(['Bream', 'Smelt'], dtype=object), array([24, 11]))

In [54]:
X_test = X[test_idx]
y_test = y[test_idx]

In [55]:
np.unique(y_test, return_counts=True)

(array(['Bream', 'Smelt'], dtype=object), array([11,  3]))